In [53]:
import pandas as pd
import numpy as np
import datetime
import math

Path = "G:/.shortcut-targets-by-id/1sZS2kL5L66960IYJ4xfNEV9CuKrs_7fE/1조/RAW DATA"

In [26]:
# df = pd.read_csv(Path+"/월별기상관측(2000.01-2021.12).csv",encoding="cp949")
# df.insert(1,'행정구역',object)

# df.to_csv(Path+"/수정_월별기상관측.csv")

In [27]:
# df = pd.read_csv(Path+"/수정_월별기상관측.csv")
# # df.head()

# # 기상 측정소 위치 라벨링
# df.loc[df['지점명']=='서울', '행정구역'] = '서울특별시'
# df.loc[df['지점명']=='무안', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='진도(첨찰산)', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='대구(기)', '행정구역'] = '대구광역시'
# df.loc[df['지점명']=='주암', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='성산포', '행정구역'] = '제주도'

In [28]:
# df_new[df_new["행정구역"]=="<class 'object'>"]["지점명"].unique()
# df.loc[df['지점명']=='주암']
# df.to_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

In [29]:
#1. 최대일사량 결측치 처리: 동일년도+동일월+동일 행정구역 데이터의 중앙값으로 대체 ex)강원도의 철원지역 결측치 2021년 1월 강원도의 최대일사량
#    (1) 세종시 : 기간 2000년 01월~2019년 04월 충청북도 최대일사량 중앙값으로 대체
 

In [30]:
# df = pd.read_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

FileNotFoundError: [Errno 2] No such file or directory: 'G:/.shortcut-targets-by-id/1sZS2kL5L66960IYJ4xfNEV9CuKrs_7fE/1조/RAW DATA/최종_월별기상관측.csv'

In [ ]:
df.drop(df.columns[:2],axis=1,inplace=True)   # 0,1번째 컬럼 drop
# df.head()

In [ ]:
# 필요 컬럼만 추출('행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)')
df_new=df[['행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]

In [ ]:
# df_ulsan = df_new[df_new["행정구역"]=="울산광역시"][["행정구역", "합계 일사량(MJ/m2)"]]

# print(len(df_ulsan))
# print(df_ulsan["합계 일사량(MJ/m2)"].isnull().sum())

# 울산광역시의 일사량은 전부 NaN 값

In [ ]:
df_new["합계 일사량(MJ/m2)"].replace(0.0, np.nan, inplace=True)   # 일사량 0.0을 전부 NaN으로 변경

In [ ]:
# df_new[df_new["합계 일사량(MJ/m2)"]==0.0]["합계 일사량(MJ/m2)"]

In [ ]:
# 일시와 행정구역에 따른 합계 일사량 중앙값
sunshine = pd.pivot_table(df_new, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [ ]:
sunshine.interpolate(inplace=True)  # 부산광역시 18년도 8월 결측치를 앞뒤 값으로 보간

In [ ]:
sunshine

In [ ]:
df2 = df_new

In [ ]:
# df["합계 일사량(MJ/m2)"].fillna()
for i in range(len(df2)) : 
    dt = df2.loc[i,"일시"]
    dstr = df2.loc[i,"행정구역"]
#     print(dt)
#     print(dstr)
    if np.isnan(df2.loc[i,"합계 일사량(MJ/m2)"]) :  # 일사량이 NaN일 때
        if df2.loc[i,"행정구역"] == "세종특별자치시" :       # 행정구역이 세종시라면
            df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["충청북도"][dt]     # 동일 일시의 충청북도 중앙값을 채움
        elif df2.loc[i,"행정구역"] == "울산광역시" :        # 행정구역이 울산광역시라면
            df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["경상남도"][dt]     # 동일 일시의 경상남도 중앙값을 채움
        else : 
            df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움

# pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [ ]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [ ]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [ ]:
# df2[df2["합계 일사량(MJ/m2)"].isnull()]
# ["울산광역시", "세종특별자치시"]

In [ ]:
avr_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="평균기온(°C)",aggfunc="median")

In [ ]:
high_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최고기온(°C)",aggfunc="median")

In [ ]:
low_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최저기온(°C)",aggfunc="median")

In [ ]:
# df2.info()

In [ ]:
def fill_tem(temperature, dataframe, pivot) : 
    for i in range(len(dataframe)) : 
        dt = dataframe.loc[i,"일시"]
        dstr = dataframe.loc[i,"행정구역"]

        if np.isnan(dataframe.loc[i,temperature]) :  # 기온 관련 컬럼값이 NaN일 때 
            dataframe.loc[i,temperature] = pivot[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움
    
    return dataframe

In [ ]:
df2 = fill_tem("최저기온(°C)", df2, low_tem)
df2 = fill_tem("최고기온(°C)", df2, high_tem)
df2 = fill_tem("평균기온(°C)", df2, avr_tem)

In [ ]:
# df2_tst[df2_tst["평균기온(°C)"].isnull()]

In [ ]:
df2['평균기온(°C)'] = df2['평균기온(°C)'].fillna(value=avr_tem["충청북도"]["2019-05"])

C:\Users\jhahn\AppData\Local\Temp\ipykernel_7476\4219879522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['평균기온(°C)'] = df2['평균기온(°C)'].fillna(value=avr_tem["충청북도"]["2019-05"])


In [ ]:
# df2.info()

In [ ]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [ ]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [ ]:
# df2_test=pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [ ]:
# df2_test

In [ ]:
chch_copy = df2[df2['행정구역']=='충청북도']
chch_copy

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
4155,충청북도,충주,2000-01,-2.9,11.2,-14.4,47.5,233.465
4156,충청북도,충주,2000-02,-3.1,8.4,-14.7,3.3,352.855
4157,충청북도,충주,2000-03,4.6,20.7,-8.9,14.5,486.280
4158,충청북도,충주,2000-04,10.1,23.2,-2.7,42.5,525.325
4159,충청북도,충주,2000-05,16.3,32.4,3.6,54.0,555.795
...,...,...,...,...,...,...,...,...
13923,충청북도,보은,2021-08,23.8,33.9,17.2,275.8,445.145
13924,충청북도,보은,2021-09,20.2,29.0,12.6,162.2,426.565
13925,충청북도,보은,2021-10,13.1,29.5,-1.7,33.8,407.200
13926,충청북도,보은,2021-11,6.3,19.9,-5.8,44.9,280.140


In [ ]:
chch_median = chch_copy.groupby("일시").median()

In [ ]:
chch_median.insert(0,column="행정구역", value="세종특별자치시")
chch_median.insert(1,column="지점명", value="세종")
# chch_median

In [ ]:
chch_median.reset_index(inplace=True)
# chch_median

,일시,행정구역,지점명,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,2000-01,세종특별자치시,세종,-2.54,9.62,-14.62,37.94,233.465
1,2000-02,세종특별자치시,세종,-2.62,8.76,-14.32,3.48,352.855
2,2000-03,세종특별자치시,세종,4.78,20.36,-8.10,17.42,486.280
3,2000-04,세종특별자치시,세종,10.60,23.66,-3.10,48.70,525.325
4,2000-05,세종특별자치시,세종,16.60,31.94,4.38,52.54,555.795
...,...,...,...,...,...,...,...,...
259,2021-08,세종특별자치시,세종,24.16,34.24,17.36,241.86,445.145
260,2021-09,세종특별자치시,세종,20.68,29.10,12.92,191.58,426.565
261,2021-10,세종특별자치시,세종,13.76,29.46,-1.00,35.26,407.200
262,2021-11,세종특별자치시,세종,6.80,19.88,-4.90,47.80,243.284


In [ ]:
# chch_median.insert(3,column="일시",value=chch_median["일시"])
# chch_median
# chch_median.columns

Index(['일시', '행정구역', '지점명', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)',
       '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)'],
      dtype='object')

In [ ]:
chch_median = chch_median[['행정구역', '지점명', '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# chch_median

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,세종특별자치시,세종,2000-01,-2.54,9.62,-14.62,37.94,233.465
1,세종특별자치시,세종,2000-02,-2.62,8.76,-14.32,3.48,352.855
2,세종특별자치시,세종,2000-03,4.78,20.36,-8.10,17.42,486.280
3,세종특별자치시,세종,2000-04,10.60,23.66,-3.10,48.70,525.325
4,세종특별자치시,세종,2000-05,16.60,31.94,4.38,52.54,555.795
...,...,...,...,...,...,...,...,...
259,세종특별자치시,세종,2021-08,24.16,34.24,17.36,241.86,445.145
260,세종특별자치시,세종,2021-09,20.68,29.10,12.92,191.58,426.565
261,세종특별자치시,세종,2021-10,13.76,29.46,-1.00,35.26,407.200
262,세종특별자치시,세종,2021-11,6.80,19.88,-4.90,47.80,243.284


In [ ]:
idx_chch = chch_median[chch_median["일시"]=="2019-05"].index[0]
# idx_chch

232

In [ ]:
chch_bef_201905 = chch_median.iloc[:idx_chch,:]
# chch_bef_201905

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,세종특별자치시,세종,2000-01,-2.54,9.62,-14.62,37.94,233.465
1,세종특별자치시,세종,2000-02,-2.62,8.76,-14.32,3.48,352.855
2,세종특별자치시,세종,2000-03,4.78,20.36,-8.10,17.42,486.280
3,세종특별자치시,세종,2000-04,10.60,23.66,-3.10,48.70,525.325
4,세종특별자치시,세종,2000-05,16.60,31.94,4.38,52.54,555.795
...,...,...,...,...,...,...,...,...
227,세종특별자치시,세종,2018-12,-1.30,14.46,-14.66,32.70,261.265
228,세종특별자치시,세종,2019-01,-1.96,8.58,-12.44,2.76,301.040
229,세종특별자치시,세종,2019-02,0.68,13.46,-11.06,30.28,340.155
230,세종특별자치시,세종,2019-03,6.24,20.32,-5.02,32.04,468.705


In [ ]:
# df2[df2["행정구역"]=="세종특별자치시"]

In [ ]:
new_df = pd.concat([df2.iloc[:14984],chch_bef_201905,df2.iloc[14984:]])
# new_df

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,속초,2000-01,-0.4,9.9,-10.9,90.2,152.990
1,강원도,속초,2000-02,-0.1,9.2,-8.6,8.4,329.070
2,강원도,속초,2000-03,6.9,18.5,-4.2,8.7,448.650
3,강원도,속초,2000-04,11.2,24.1,2.4,23.4,486.355
4,강원도,속초,2000-05,14.3,30.5,6.2,71.0,520.280
...,...,...,...,...,...,...,...,...
22841,경상남도,남해,2021-08,26.1,35.5,20.7,473.2,481.070
22842,경상남도,남해,2021-09,22.2,30.1,16.3,164.6,366.930
22843,경상남도,남해,2021-10,17.2,28.9,4.7,43.6,429.240
22844,경상남도,남해,2021-11,10.7,21.7,1.4,53.5,323.960


In [ ]:
new_df[new_df["행정구역"]=="세종특별자치시"]

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,세종특별자치시,세종,2000-01,-2.54,9.62,-14.62,37.94,233.465
1,세종특별자치시,세종,2000-02,-2.62,8.76,-14.32,3.48,352.855
2,세종특별자치시,세종,2000-03,4.78,20.36,-8.10,17.42,486.280
3,세종특별자치시,세종,2000-04,10.60,23.66,-3.10,48.70,525.325
4,세종특별자치시,세종,2000-05,16.60,31.94,4.38,52.54,555.795
...,...,...,...,...,...,...,...,...
15011,세종특별자치시,세종,2021-08,25.10,34.50,18.60,232.10,445.145
15012,세종특별자치시,세종,2021-09,21.80,29.40,14.40,184.60,426.565
15013,세종특별자치시,세종,2021-10,15.00,29.80,0.70,36.40,407.200
15014,세종특별자치시,세종,2021-11,8.20,20.90,-2.40,60.90,280.140


In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23078 entries, 0 to 22845
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                23078 non-null  object 
 1   지점명                 23078 non-null  object 
 2   일시                  23078 non-null  object 
 3   평균기온(°C)            23078 non-null  float64
 4   최고기온(°C)            23078 non-null  float64
 5   최저기온(°C)            23078 non-null  float64
 6   월합강수량(00~24h만)(mm)  23052 non-null  float64
 7   합계 일사량(MJ/m2)       23078 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1.6+ MB


In [ ]:
new_df[new_df['행정구역']=='세종특별자치시'].isnull().sum()

행정구역                  0
지점명                   0
일시                    0
평균기온(°C)              0
최고기온(°C)              0
최저기온(°C)              0
월합강수량(00~24h만)(mm)    0
합계 일사량(MJ/m2)         0
dtype: int64

In [ ]:
new_df.fillna(0.0,inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23078 entries, 0 to 22845
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   행정구역                23078 non-null  object 
 1   지점명                 23078 non-null  object 
 2   일시                  23078 non-null  object 
 3   평균기온(°C)            23078 non-null  float64
 4   최고기온(°C)            23078 non-null  float64
 5   최저기온(°C)            23078 non-null  float64
 6   월합강수량(00~24h만)(mm)  23078 non-null  float64
 7   합계 일사량(MJ/m2)       23078 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1.6+ MB


In [ ]:
# new_df.to_csv("data/최종_최종__월별기상관측.csv",encoding="cp949")

In [ ]:
new_df.groupby(["행정구역", "일시"]).median()

평균기온(°C)  최고기온(°C)  최저기온(°C)  월합강수량(00~24h만)(mm)  합계 일사량(MJ/m2)
행정구역 일시                                                                      
강원도  2000-01      -2.9       8.5     -16.0                48.1        152.990
     2000-02      -3.3       7.7     -15.8                 5.7        329.070
     2000-03       4.4      19.3      -8.6                14.1        448.650
     2000-04      10.4      23.7      -3.1                37.1        486.355
     2000-05      15.9      31.6       3.9                75.2        520.280
...                ...       ...       ...                 ...            ...
충청북도 2021-08      23.8      33.9      17.5               239.9        445.145
     2021-09      20.2      29.0      12.6               185.0        426.565
     2021-10      13.5      29.5      -1.7                33.8        407.200
     2021-11       6.4      19.9      -5.2                44.9        280.140
     2021-12      -0.1      12.6     -14.7                 5.6        249.960

[4488 rows x 5 columns]

In [7]:
new_df = pd.read_csv("./data/최종_최종__월별기상관측.csv", encoding="cp949")
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23078 entries, 0 to 23077
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          23078 non-null  int64  
 1   행정구역                23078 non-null  object 
 2   지점명                 23078 non-null  object 
 3   일시                  23078 non-null  object 
 4   평균기온(°C)            23078 non-null  float64
 5   최고기온(°C)            23078 non-null  float64
 6   최저기온(°C)            23078 non-null  float64
 7   월합강수량(00~24h만)(mm)  23078 non-null  float64
 8   합계 일사량(MJ/m2)       23078 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 1.6+ MB


In [9]:
new_df.drop("Unnamed: 0", axis=1, inplace=True)  # 0번째 컬럼 drop
# new_df.head(3)

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,속초,2000-01,-0.4,9.9,-10.9,90.2,152.99
1,강원도,속초,2000-02,-0.1,9.2,-8.6,8.4,329.07
2,강원도,속초,2000-03,6.9,18.5,-4.2,8.7,448.65


In [13]:
new_df["일시"] = pd.to_datetime(new_df["일시"])  # 일시를 datetime으로 변경
# new_df.head()

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,속초,2000-01-01,-0.4,9.9,-10.9,90.2,152.990
1,강원도,속초,2000-02-01,-0.1,9.2,-8.6,8.4,329.070
2,강원도,속초,2000-03-01,6.9,18.5,-4.2,8.7,448.650
3,강원도,속초,2000-04-01,11.2,24.1,2.4,23.4,486.355
4,강원도,속초,2000-05-01,14.3,30.5,6.2,71.0,520.280


In [15]:
new_df["연도"] =  new_df["일시"].dt.year   # 연도만 추출
new_df["월"] =  new_df["일시"].dt.month    # 월만 추출
# new_df.head(10)

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),연도,월
0,강원도,속초,2000-01-01,-0.4,9.9,-10.9,90.2,152.990,2000,1
1,강원도,속초,2000-02-01,-0.1,9.2,-8.6,8.4,329.070,2000,2
2,강원도,속초,2000-03-01,6.9,18.5,-4.2,8.7,448.650,2000,3
3,강원도,속초,2000-04-01,11.2,24.1,2.4,23.4,486.355,2000,4
4,강원도,속초,2000-05-01,14.3,30.5,6.2,71.0,520.280,2000,5
5,강원도,속초,2000-06-01,19.2,33.3,9.7,175.7,547.195,2000,6
6,강원도,속초,2000-07-01,24.8,33.6,17.0,109.2,512.230,2000,7
7,강원도,속초,2000-08-01,24.0,33.7,17.9,440.3,412.735,2000,8
8,강원도,속초,2000-09-01,18.6,32.0,11.2,347.2,347.645,2000,9
9,강원도,속초,2000-10-01,14.5,23.0,5.1,7.0,311.070,2000,10


In [16]:
# new_df.columns

Index(['행정구역', '지점명', '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)',
       '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)', '연도', '월'],
      dtype='object')

In [17]:
# 컬럼 순서 변경
new_df = new_df[['행정구역', '지점명', '연도', '월', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# new_df.head(3)


,행정구역,지점명,연도,월,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,속초,2000,1,-0.4,9.9,-10.9,90.2,152.99
1,강원도,속초,2000,2,-0.1,9.2,-8.6,8.4,329.07
2,강원도,속초,2000,3,6.9,18.5,-4.2,8.7,448.65


In [30]:
test = new_df.groupby(["연도","행정구역","월"]).median()[:24]

In [31]:
test

평균기온(°C)  최고기온(°C)  최저기온(°C)  월합강수량(00~24h만)(mm)  합계 일사량(MJ/m2)
연도   행정구역 월                                                                  
2000 강원도  1      -2.90      8.50    -16.00               48.10        152.990
          2      -3.30      7.70    -15.80                5.70        329.070
          3       4.40     19.30     -8.60               14.10        448.650
          4      10.40     23.70     -3.10               37.10        486.355
          5      15.90     31.60      3.90               75.20        520.280
          6      21.30     33.60      9.60              116.90        547.195
          7      24.90     34.30     16.70              153.40        512.230
          8      24.10     33.70     17.40              348.00        412.735
          9      18.50     31.10      8.50              256.90        347.645
          10     12.10     26.10     -1.60               16.90        311.070
          11      4.10     20.30     -8.40               30.20        231.670
          12     -1.40     12.50    -14.90               16.90        234.160
     경기도  1      -2.65     10.20    -15.00               52.20        202.280
          2      -2.30      8.30    -15.05                1.55        317.510
          3       5.25     19.75     -8.60                2.55        455.280
          4      10.85     23.70     -2.20               23.55        530.950
          5      17.05     31.40      5.55               67.55        562.770
          6      22.70     33.35     12.65              113.60        554.200
          7      25.75     34.05     18.80              244.00        542.240
          8      25.00     34.00     18.25              417.40        453.020
          9      19.00     31.20      9.20              160.85        394.070
          10     12.60     26.80     -1.70               18.55        359.460
          11      4.70     20.75     -7.90               25.00        262.150
          12     -0.55     13.25    -12.90               24.10        248.430

In [37]:
test.iloc[[1,13]]

평균기온(°C)  최고기온(°C)  최저기온(°C)  월합강수량(00~24h만)(mm)  합계 일사량(MJ/m2)
연도   행정구역 월                                                                 
2000 강원도  2      -3.3       7.7    -15.80                5.70         329.07
     경기도  2      -2.3       8.3    -15.05                1.55         317.51

In [40]:
# test2 = test.iloc[[0, 12]]
# for i in range(1, int(len(test)/2)) : 
#     a = test.iloc[[i,i+12]]
#     # print(a)
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer")
# test2

In [41]:
# test2 = test.iloc[0]
# for i in range(1, len(test)) : 
#     a = test.iloc[i]
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer") 
# test2

In [44]:
empty_array = np.empty((2,60))
empty_array[:] = np.nan
empty_array

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan]])

In [46]:
empty_frame = pd.DataFrame(empty_array)
empty_frame.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
new_df.columns

Index(['행정구역', '지점명', '연도', '월', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)',
       '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)'],
      dtype='object')

In [50]:
cname = []
for i in range(1,13) : 
    L = ['평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']
    for elem in L : 
        word = elem + "_{}월".format(i)
        cname.append(word)
cname

['평균기온(°C)_1월',
 '최고기온(°C)_1월',
 '최저기온(°C)_1월',
 '월합강수량(00~24h만)(mm)_1월',
 '합계 일사량(MJ/m2)_1월',
 '평균기온(°C)_2월',
 '최고기온(°C)_2월',
 '최저기온(°C)_2월',
 '월합강수량(00~24h만)(mm)_2월',
 '합계 일사량(MJ/m2)_2월',
 '평균기온(°C)_3월',
 '최고기온(°C)_3월',
 '최저기온(°C)_3월',
 '월합강수량(00~24h만)(mm)_3월',
 '합계 일사량(MJ/m2)_3월',
 '평균기온(°C)_4월',
 '최고기온(°C)_4월',
 '최저기온(°C)_4월',
 '월합강수량(00~24h만)(mm)_4월',
 '합계 일사량(MJ/m2)_4월',
 '평균기온(°C)_5월',
 '최고기온(°C)_5월',
 '최저기온(°C)_5월',
 '월합강수량(00~24h만)(mm)_5월',
 '합계 일사량(MJ/m2)_5월',
 '평균기온(°C)_6월',
 '최고기온(°C)_6월',
 '최저기온(°C)_6월',
 '월합강수량(00~24h만)(mm)_6월',
 '합계 일사량(MJ/m2)_6월',
 '평균기온(°C)_7월',
 '최고기온(°C)_7월',
 '최저기온(°C)_7월',
 '월합강수량(00~24h만)(mm)_7월',
 '합계 일사량(MJ/m2)_7월',
 '평균기온(°C)_8월',
 '최고기온(°C)_8월',
 '최저기온(°C)_8월',
 '월합강수량(00~24h만)(mm)_8월',
 '합계 일사량(MJ/m2)_8월',
 '평균기온(°C)_9월',
 '최고기온(°C)_9월',
 '최저기온(°C)_9월',
 '월합강수량(00~24h만)(mm)_9월',
 '합계 일사량(MJ/m2)_9월',
 '평균기온(°C)_10월',
 '최고기온(°C)_10월',
 '최저기온(°C)_10월',
 '월합강수량(00~24h만)(mm)_10월',
 '합계 일사량(MJ/m2)_10월',
 '평균기온(°C)_11월',
 '최고기온(°C)_11월',
 '최저기온(°C)_

In [52]:
empty_frame.columns = cname
empty_frame

,평균기온(°C)_1월,최고기온(°C)_1월,최저기온(°C)_1월,월합강수량(00~24h만)(mm)_1월,합계 일사량(MJ/m2)_1월,평균기온(°C)_2월,최고기온(°C)_2월,최저기온(°C)_2월,월합강수량(00~24h만)(mm)_2월,합계 일사량(MJ/m2)_2월,...,평균기온(°C)_11월,최고기온(°C)_11월,최저기온(°C)_11월,월합강수량(00~24h만)(mm)_11월,합계 일사량(MJ/m2)_11월,평균기온(°C)_12월,최고기온(°C)_12월,최저기온(°C)_12월,월합강수량(00~24h만)(mm)_12월,합계 일사량(MJ/m2)_12월
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(len(test)) : 
    j = math.floor(i/12)
    k = math.floor(i/5)
    empty_frame[]